In [1]:
import nibabel as nib
import numpy as np
from pathlib import Path

def load_nii_slices(folder_path, max_slices):
    all_slices = []
    for nii_path in sorted(folder_path.glob("*.nii")):
        print(f"Loading {nii_path.name}")
        img = nib.load(nii_path)
        data = img.get_fdata()  # shape: (192, 192, N)

        assert data.shape[0:2] == (192, 192), f"Unexpected shape: {data.shape}"

        for i in range(data.shape[2]):
            slice_2d = data[:, :, i]
            slice_2d = np.expand_dims(slice_2d, axis=-1)
            all_slices.append(slice_2d)

            if len(all_slices) >= max_slices:
                print(f"Reached max slice limit: {max_slices}")
                return np.stack(all_slices, axis=0)

    return np.stack(all_slices, axis=0)

# Define paths
ct_path = Path("TumSeg_separate/CT")
staple_path = Path("TumSeg_separate/STAPLE")

# Load up to 1000 slices
x = load_nii_slices(ct_path, max_slices=1000)
y = load_nii_slices(staple_path, max_slices=1000)

ValueError: need at least one array to stack

In [ ]:
print("X shape:", x.shape)
print("Y shape:", y.shape)

In [ ]:
import numpy as np

# Calculate the mean of each 192x192 image (ignoring the last channel dimension)
avg_values = np.mean(y, axis=(1, 2))  # shape: (1000,)

print("Average intensity values for all 1000 images:")
print(avg_values)
print("Shape of avg_values:", avg_values.shape)


In [ ]:
# Find indices where average intensity is NOT zero
non_zero_indices = np.where(avg_values != 0)[0]
zero_indices = np.where(avg_values == 0)[0]

print("Indices with non-zero average intensity:")
print(non_zero_indices)
print("Total non-zero images:", len(non_zero_indices))

print("Indices with zero average intensity:")
print(zero_indices)
print("Total non-zero images:", len(zero_indices))


In [ ]:
temp=len(non_zero_indices)
zero_indices = zero_indices[:temp]
print(zero_indices.shape)
print(non_zero_indices.shape)
print(zero_indices)
print(non_zero_indices)

In [ ]:
indices = np.concatenate([zero_indices, non_zero_indices])
indices = np.sort(indices)
print(indices)
print(indices.shape)

In [ ]:
X_fresh = x[indices]
Y_fresh = y[indices]

print(X_fresh.shape)
print(Y_fresh.shape)

# Model Training

In [ ]:
import matplotlib.pyplot as plt

# Choose the index you want to visualize
i = 152  # or any number < 1000

# Remove channel dimension for display
image = X_fresh[i, :, :, 0]
label = Y_fresh[i, :, :, 0]

# Plot side-by-side
plt.figure(figsize=(10, 5))

plt.subplot(1, 2, 1)
plt.imshow(image, cmap='gray')
plt.title(f"CT Slice - X[{i}]")
plt.axis('off')

plt.subplot(1, 2, 2)
plt.imshow(label, cmap='gray')
plt.title(f"STAPLE Mask - Y[{i}]")
plt.axis('off')

plt.tight_layout()
plt.show()


In [ ]:
X = np.expand_dims(X_fresh, -1)
Y = np.expand_dims(Y_fresh, -1)
print("X shape:", X.shape)
print("Y shape:", Y.shape)

In [ ]:
from sklearn.model_selection import train_test_split

# Perform train-test split
X_train, X_test, y_train, Y_test = train_test_split(X, Y, test_size=0.15, random_state=32)

# Perform train-validation split
X_train, X_val, Y_train, Y_val = train_test_split(X_train, y_train, test_size=0.15, random_state=32)

# Print shapes to verify the split
print('X_train shape:', X_train.shape)
print('y_train shape:', y_train.shape)
print('X_test shape:', X_test.shape)
print('y_test shape:', Y_test.shape)
print('X_test shape:', X_val.shape)
print('y_test shape:', Y_val.shape)

# U NET 

In [ ]:
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Conv2DTranspose, concatenate

# Encoder block: two conv layers followed by max pooling
def encoder_block(input, num_filters):
    conv = Conv2D(num_filters, (3, 3), activation="relu", padding="same")(input)
    conv = Conv2D(num_filters, (3, 3), activation="relu", padding="same")(conv)
    pool = MaxPooling2D((2, 2))(conv)
    return conv, pool

# Decoder block: transpose conv, skip connection, then conv block
def decoder_block(input, skip_features, num_filters):
    uconv = Conv2DTranspose(num_filters, (2, 2), strides=2, padding="same")(input)
    con = concatenate([uconv, skip_features])
    conv = conv_block(con, num_filters)
    return conv

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Convolution block used in bottleneck and decoder
def conv_block(input, num_filters):
    conv = Conv2D(num_filters, (3, 3), activation="relu", padding="same")(input)
    conv = Conv2D(num_filters, (3, 3), activation="relu", padding="same")(conv)
    return conv

# Build the full U-Net model
def build_model(input_shape):
    input_layer = Input(input_shape)
    
    s1, p1 = encoder_block(input_layer, 64)
    s2, p2 = encoder_block(p1, 128)
    s3, p3 = encoder_block(p2, 256)
    s4, p4 = encoder_block(p3, 512)

    b1 = conv_block(p4, 1024)

    d1 = decoder_block(b1, s4, 512)
    d2 = decoder_block(d1, s3, 256)
    d3 = decoder_block(d2, s2, 128)
    d4 = decoder_block(d3, s1, 64)
    
    output_layer = Conv2D(1, 1, padding="same", activation="sigmoid")(d4)
    
    model = Model(input_layer, output_layer, name="U-Net")
    return model

# Instantiate and compile the model
model = build_model(input_shape=(192, 192, 1))
model.compile(loss="binary_crossentropy", optimizer="Adam", metrics=["accuracy"])
model.summary()

In [ ]:
history = model.fit(X_train, Y_train, epochs=2, batch_size=32, validation_data=(X_val, Y_val))

In [ ]:
import matplotlib.pyplot as plt

# Suppose this is your training history
# history = model.fit(...)

# Plot Accuracy
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Val Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

# Plot Loss
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()


In [ ]:
# X_val and Y_val are your validation images and masks
loss, accuracy = model.evaluate(X_test, Y_test, verbose=0)
print("Final Accuracy:", accuracy)


In [ ]:
model.save('unet_model_Fresh.h5')


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Create subplot grid
fig, ax = plt.subplots(5, 3, figsize=(10, 18))

# Randomly select 5 test samples
indices = np.random.randint(0, X_test.shape[0], 5)

for i in range(5):
    idx = indices[i]
    
    # Remove extra dimensions: (192, 192)
    image = np.squeeze(X_test[idx])
    mask = np.squeeze(Y_test[idx])
    
    # Predict and squeeze
    pred = model.predict(np.expand_dims(X_test[idx], axis=0), verbose=0)[0]
    pred = np.squeeze(pred) > 0.5  # Apply threshold if binary segmentation

    # Plot image
    ax[i, 0].imshow(image, cmap='gray')
    ax[i, 0].set_title('Image')
    ax[i, 0].axis('off')
    
    # Plot ground truth mask
    ax[i, 1].imshow(mask, cmap='gray')
    ax[i, 1].set_title('Ground Truth')
    ax[i, 1].axis('off')
    
    # Plot predicted mask
    ax[i, 2].imshow(pred, cmap='gray')
    ax[i, 2].set_title('Prediction')
    ax[i, 2].axis('off')

# Overall title
fig.suptitle('U-Net Predictions on Test Set', fontsize=16)
plt.tight_layout()
plt.show()
